# ML ASSIGNMENT LAB 07
PRAKRUTHI B GOPAL 21BDA25

3.Fit a CNN model on the dataset which has been assigned to you. Print a classification report to see the model metrics on train and test datasets.

In [2]:
import os
import numpy as np
import tensorflow as tf

my_random_seed = 1331
np.random.seed(my_random_seed)
tf.random.set_seed(my_random_seed)

from keras.models import Model, load_model
from keras.layers import Input, Conv2D, Dense, Flatten
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [7]:
'''
Configurable parameters
'''
model_path = 'model_test.h5'
in_dir = 'C:/Users/91866/Downloads/archive (4)/snake-eyes'
batch_size = 512
epochs = 3

In [8]:
'''
Load data into memory
'''
def _read_vectors(filename):
    return np.fromfile(filename, dtype=np.uint8).reshape(-1, 401)


# ------ Setup training and validation sets ----------
snk = np.vstack(tuple(_read_vectors(
    os.path.join(in_dir, "snakeeyes_{:02d}.dat".format(nn))) for nn in range(10)))

x = snk[:, 1:]
y_ = snk[:, 0]

# Convert labels to one hot
y = np.zeros([len(y_), 12], dtype=bool)
for i in range(len(y_)):
    label = y_[i]
    y[i, label - 1] = True

x_train, x_validation, y_train, y_validation = \
    train_test_split(x, y, test_size=0.1, random_state=42, stratify=y)

mean = x_train.mean(axis=0)  # Compute data statistics on training data only
std = x_train.std(axis=0)

# Normalize data to be zero mean and unit variance
x_train = (x_train - mean)/std
x_validation = (x_validation - mean)/std

# Reshape
x_train = x_train.reshape(-1, 20, 20, 1)
x_validation = x_validation.reshape(-1, 20, 20, 1)

# ------ Setup test set ----------
snk_test = _read_vectors(os.path.join(in_dir, "snakeeyes_test.dat"))
x_test = snk_test[:, 1:]
y_test_ = snk_test[:, 0]

# Convert labels to one hot
y_test = np.zeros([len(y_test_), 12], dtype=bool)
for i in range(len(y_test)):
    label = y_test_[i]
    y_test[i, label - 1] = True

x_test = (x_test - mean)/std  # Normalize
x_test = x_test.reshape(-1, 20, 20, 1)  # Reshape

In [9]:
'''
Get Model instance
''' 
# Define model
inputs = Input(shape=(20, 20, 1))
x = Conv2D(2, 5, padding='same', activation='relu')(inputs)
x = Conv2D(8, 5, padding='same', activation='relu')(x)
x = Conv2D(8, 5, padding='same', activation='relu')(x)
x = Conv2D(8, 5, padding='same', activation='relu')(x)
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
x = Dense(12, activation='softmax')(x)
outputs = x

m = Model(inputs=inputs, outputs=outputs)
print(m.summary())    

opt = RMSprop(lr=0.001)
m.compile(optimizer=opt, metrics=['accuracy'], loss='categorical_crossentropy')
cb_chkpt = ModelCheckpoint(model_path, monitor='val_loss', verbose=0, save_best_only=True)

m.fit(x=x_train, y=y_train,
           batch_size=batch_size,
           epochs=epochs,
           validation_data=(x_validation, y_validation),
           callbacks=[cb_chkpt])    

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20, 20, 1)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 20, 20, 2)         52        
                                                                 
 conv2d_5 (Conv2D)           (None, 20, 20, 8)         408       
                                                                 
 conv2d_6 (Conv2D)           (None, 20, 20, 8)         1608      
                                                                 
 conv2d_7 (Conv2D)           (None, 20, 20, 8)         1608      
                                                                 
 flatten_1 (Flatten)         (None, 3200)              0         
                                                                 
 dense_4 (Dense)             (None, 32)                1024

C:\Users\91866\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/3
1758/1758 [==============================] - 685s 388ms/step - loss: 1.6325 - accuracy: 0.3210 - val_loss: 1.4306 - val_accuracy: 0.3691
Epoch 2/3
1758/1758 [==============================] - 584s 332ms/step - loss: 1.0563 - accuracy: 0.5658 - val_loss: 0.8460 - val_accuracy: 0.6584
Epoch 3/3
1758/1758 [==============================] - 580s 330ms/step - loss: 0.6775 - accuracy: 0.7291 - val_loss: 0.6144 - val_accuracy: 0.7458


In [10]:
'''
Evaluate model performance
'''
test_loss, test_accuracy = m.evaluate(x_test, y_test)
print('Test loss: {:} Test Accuracy: {:}'.format(test_loss, test_accuracy))

313/313 [==============================] - 2s 7ms/step - loss: 0.6040 - accuracy: 0.7462
Test loss: 0.6040034890174866 Test Accuracy: 0.7462000250816345


1.Have you come across Grid Search Cross Validation? Fit any two models covered in previous classes and optimize them using Grid search CV.

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython import get_ipython
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score

In [12]:
data = pd.read_csv("C:/Users/91866/Downloads/online_education.csv")

In [13]:
data.head()


,Education Level,Institution Type,Gender,Age,Device,IT Student,Location,Financial Condition,Internet Type,Network Type,Flexibility Level
0,University,Private,Male,23,Tab,No,Town,Mid,Wifi,4G,Moderate
1,University,Private,Female,23,Mobile,No,Town,Mid,Mobile Data,4G,Moderate
2,College,Public,Female,18,Mobile,No,Town,Mid,Wifi,4G,Moderate
3,School,Private,Female,11,Mobile,No,Town,Mid,Mobile Data,4G,Moderate
4,School,Private,Female,18,Mobile,No,Town,Poor,Mobile Data,3G,Low


In [14]:
data.isnull().sum()

Education Level        0
Institution Type       0
Gender                 0
Age                    0
Device                 0
IT Student             0
Location               0
Financial Condition    0
Internet Type          0
Network Type           0
Flexibility Level      0
dtype: int64

In [15]:
data.describe()

,Age
count,1205.000000
mean,17.065560
std,5.830369
min,9.000000
25%,11.000000
50%,18.000000
75%,23.000000
max,27.000000


In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [17]:
FlexibilityLevel = le.fit_transform(data['Flexibility Level'])
data['Flexibility Level'] = FlexibilityLevel

In [18]:
categorical_features = ['Education Level','Gender','Institution Type',
'Device','IT Student','Location',
'Financial Condition','Internet Type',
'Network Type']

In [19]:
EducationLevel= le.fit_transform(data['Education Level'])
Gender = le.fit_transform(data['Gender'])
InstitutionType = le.fit_transform(data['Institution Type'])
Device = le.fit_transform(data['Device'])
ITStudent= le.fit_transform(data['IT Student'])
Location = le.fit_transform(data['Location'])
FinancialCondition = le.fit_transform(data['Financial Condition'])
InternetType = le.fit_transform(data['Internet Type'])
NetworkType = le.fit_transform(data['Network Type'])

In [20]:
data['Education Level'] = EducationLevel
data['Gender'] = Gender
data['Institution Type'] = InstitutionType
data['Device'] = Device
data['IT Student'] = ITStudent
data['Location'] = Location
data['Financial Condition'] = FinancialCondition
data['Internet Type'] = InternetType
data['Network Type'] = NetworkType

In [21]:
x = data.drop('Flexibility Level',axis=1)
y = data['Flexibility Level']

In [22]:
x.shape


(1205, 10)

In [23]:
y.shape


(1205,)

In [24]:
from sklearn.preprocessing import StandardScaler


In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y,
test_size=0.30,
random_state=42)


In [26]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


In [27]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train,y_train)

DecisionTreeClassifier()

In [28]:
y_pred = model.predict(X_test)

In [29]:
print("Training Accuracy :", model.score(X_train, y_train))
print("Testing Accuracy :", model.score(X_test, y_test))

Training Accuracy : 0.8623962040332147
Testing Accuracy : 0.7734806629834254


In [30]:
from sklearn.metrics import classification_report

In [31]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.51      0.55        39
           1       0.81      0.82      0.82       154
           2       0.77      0.79      0.78       169

    accuracy                           0.77       362
   macro avg       0.73      0.71      0.72       362
weighted avg       0.77      0.77      0.77       362



In [32]:
from sklearn.ensemble import RandomForestClassifier
classifier= RandomForestClassifier(n_estimators= 10,
criterion="entropy")

In [33]:
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [34]:
y_pred = classifier.predict(X_test)

In [35]:
print("Training Accuracy :", classifier.score(X_train, y_train))
print("Testing Accuracy :", classifier.score(X_test, y_test))

Training Accuracy : 0.8623962040332147
Testing Accuracy : 0.7762430939226519


In [36]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.46      0.62        39
           1       0.81      0.78      0.79       154
           2       0.73      0.85      0.79       169

    accuracy                           0.78       362
   macro avg       0.83      0.70      0.73       362
weighted avg       0.79      0.78      0.77       362



In [37]:
classifier.feature_importances_

array([0.09113421, 0.08894727, 0.09670662, 0.18851859, 0.07255028,
       0.05647213, 0.0721868 , 0.16886479, 0.08182668, 0.08279263])

In [38]:
grid_param = {
'criterion': ['gini', 'entropy'],
'max_depth' : range(2,32,1),
'min_samples_leaf' : range(1,10,1),
'min_samples_split': range(2,10,1),
'splitter' : ['best', 'random']
}

In [39]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator=model,
param_grid=grid_param,
cv=5,
n_jobs =2, verbose=1)

In [40]:
grid_search.fit(X_train,y_train)


Fitting 5 folds for each of 8640 candidates, totalling 43200 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=2,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 32),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10),
                         'splitter': ['best', 'random']},
             verbose=1)

In [41]:
best_parameters = grid_search.best_params_
print(best_parameters)

{'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 3, 'splitter': 'random'}


In [42]:
grid_search.best_score_

0.795942519019442

In [43]:
clf = DecisionTreeClassifier(criterion = 'gini', max_depth = 25,
min_samples_leaf= 1, min_samples_split= 2,
splitter ='best')
clf.fit(X_train,y_train)


DecisionTreeClassifier(max_depth=25)

In [54]:
ac2_clf= clf.score(X_test ,y_test)
ac2_clf

0.6408839779005525

In [55]:
grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy'],
    'random_state' : [18]
}

In [57]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator=classifier,
param_grid=grid,
cv=5,
n_jobs =2, verbose=1)

In [59]:
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(criterion='entropy',
                                              n_estimators=10),
             n_jobs=2,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [200, 300, 400, 500],
                         'random_state': [18]},
             verbose=1)

In [60]:
best_parameters = grid_search.best_params_
print(best_parameters)

{'criterion': 'entropy', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 300, 'random_state': 18}


In [61]:
clf = RandomForestClassifier(n_estimators = 200, max_depth=4, max_features='sqrt' , random_state = 18).fit(X_train, y_train)

In [62]:
ac2_clf= clf.score(X_test ,y_test)
ac2_clf

0.6519337016574586